In [1]:
import numpy as np
import pandas as pd
import openpyxl as xl
import FILibExcel
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb 
import shap

/Users/alexglushko/ProgProjects/ChemML/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
excel_path = r"../../static/mock/antibio_data/20.02.2024 AntiBio_дополнено.xlsx"
artifacts_path = r"../../static/mock/antibio_artifacts/20-02/"

In [18]:
tables_dict = FILibExcel.get_all_tables(file_name=excel_path)
df_compounds = tables_dict["CompoundsData"]['dataframe']
df_bacterias = tables_dict["BacteriasData"]['dataframe']
df_experiments = tables_dict["ExperimentsData"]['dataframe']


worksheet name: Table
tables in worksheet: 0

worksheet name: CompoundsData
tables in worksheet: 1
table name: CompoundsData

worksheet name: BacteriasData
tables in worksheet: 1
table name: BacteriasData

worksheet name: pH and concentration
tables in worksheet: 0

worksheet name: ExperimentsData
tables in worksheet: 1
table name: ExperimentsData

worksheet name: Legend-Main
tables in worksheet: 0

worksheet name: Bac
tables in worksheet: 0


In [38]:
# One hot encoding
types_num = 9
for i in range(1, types_num + 1):
    df_compounds[f'Crystal structure type {i}'] = 0

for index, _ in df_compounds.iterrows():
    item = df_compounds.loc[index]
    if item['Crystal structure type'] is None:
        if item['Crystal structure type (literat.)'].:
            print(item)
            i = int(item['Crystal structure type (literat.)'])
            item[f'Crystal structure type {i}'] = 1
        continue
        

    for i in str(item['Crystal structure type']).split(';'):
        item[f'Crystal structure type {int(i)}'] = 1
df_experiments.head

REE                                         Eu
Ionic radii                                NaN
1st Ionization energy                      NaN
2nd Ionization energy                      NaN
3rd Ionization energy                      NaN
4th Ionization energy                      NaN
Crystal structure type                    None
Crystal structure type (literat.)          3.0
(salt)  d1-1H2O                          2.404
(salt)  d1-2H2O                          2.419
(salt)  d1-3H2O                          2.438
(salt) d1-4H2O                           2.497
(salt)  d1-5H2O                            NaN
(salt) d1-1NO3                           2.467
(salt) d1-2NO3                           2.497
(salt) d1-3NO3                           2.516
(salt) d1-4NO3                           2.547
(salt) d1-5NO3                           2.571
(salt) d1-6NO3                           2.633
(salt) d1                                2.499
(salt) d2-1NO3                           2.944
(salt) d2-2NO

/var/folders/03/8g_v72pd32vfbhfq91cfy1780000gn/T/ipykernel_19779/1800518156.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item[f'Crystal structure type {int(i)}'] = 1
/var/folders/03/8g_v72pd32vfbhfq91cfy1780000gn/T/ipykernel_19779/1800518156.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item[f'Crystal structure type {i}'] = 1


ValueError: cannot convert float NaN to integer

In [34]:
df = df_experiments.merge(df_compounds, left_on='REE', right_on='REE').merge(df_bacterias, left_on='Bacteria', right_on='Name')
df.head().columns

Index(['REE', 'Bacteria', 'Is salt', 'Stunting diameter', 'Ionic radii',
       '1st Ionization energy', '2nd Ionization energy',
       '3rd Ionization energy', '4th Ionization energy',
       'Crystal structure type', 'Crystal structure type (literat.)',
       '(salt)  d1-1H2O', '(salt)  d1-2H2O  ', '(salt)  d1-3H2O  ',
       '(salt) d1-4H2O', '(salt)  d1-5H2O ', '(salt) d1-1NO3',
       '(salt) d1-2NO3 ', '(salt) d1-3NO3  ', '(salt) d1-4NO3    ',
       '(salt) d1-5NO3      ', '(salt) d1-6NO3   ', '(salt) d1',
       '(salt) d2-1NO3', '(salt) d2-2NO3  ', '(salt) d2-3NO3     ',
       '(salt) d3-1H2O', '(salt) d3-2H2O   ',
       'Coordination number (solid) Ln-O', 'Coordination number (solid) Ln-N',
       'x', 'n', 'Electronegativity, Oganov', 'Electronegativity, Malliken',
       'Electronegativity, Poland', 'Electronegativity, Allred-Rochow',
       'R1 (Ln-O)', 'R1 (Ln-N)', 'R2 (Ln-O)',
       'Coordination number solution 1(Ln-O)',
       'Coordination number solution  1(Ln-N

In [20]:
df = df.drop(['REE', 'Bacteria', 'Name', 'Electronic structure RE3+', 'Price', 'Name'], axis=1)

In [21]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 56 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Is salt                                245 non-null    int64  
 1   Stunting diameter                      245 non-null    int64  
 2   Ionic radii                            181 non-null    float64
 3   1st Ionization energy                  181 non-null    float64
 4   2nd Ionization energy                  181 non-null    float64
 5   3rd Ionization energy                  181 non-null    float64
 6   4th Ionization energy                  181 non-null    float64
 7   Crystal structure type                 175 non-null    object 
 8   Crystal structure type (literat.)      227 non-null    float64
 9   (salt)  d1-1H2O                        227 non-null    float64
 10  (salt)  d1-2H2O                        227 non-null    float64
 11  (salt)